# Перед первым использованием установите библиотеки

In [79]:
# !pip install pandas
# !pip install numpy
# !pip install pytest-warnings
# !pip install contextlib2
# !pip install psycopg2
# !pip install SQLAlchemy

## Навигация
- **[Перечень всех кодов и названий продуктов](#Перечень-всех-кодов-и-названий-продуктов)**
- **[Преобразование параметра qty датафрейма](#Преобразование-параметра-qty-датафрейма)**

In [1]:
import pandas as pd
import numpy as np
import os

import warnings
warnings.simplefilter('ignore') 

# Для автоматического закрытия курсора
from contextlib import closing
import psycopg2
from psycopg2 import Error
from sqlalchemy import create_engine

import json
from datetime import datetime
import time


# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)

In [16]:
# Считываем конфиг названия таблиц и базы данных
with open('config_js.json', 'r') as fl:
    name_table_or_bd = json.load(fl)

In [2]:
# Инициализация подключений для работы с БД
engine = psycopg2.connect(user=os.getenv('USER'),
                          # пароль, который указали при установке PostgreSQL
                          password=os.getenv('PASSWORD'),
                          host=os.getenv('HOST'),
                          port=os.getenv('PORT'),
                          database=os.getenv('DATABASE'))
conn = create_engine('postgresql://{}:{}@{}:{}/{}'
                     .format(os.getenv('USER'), os.getenv('PASSWORD'), os.getenv('HOST'), os.getenv('PORT'), os.getenv('DATABASE')))


In [3]:
# Считываем датафреймы
df_import = pd.read_csv('india_import.csv')
df_export = pd.read_csv('india_export.csv')

In [4]:
# Объединяем импорт и экспорт
df_full_flow = pd.concat((df_import, df_export))

In [5]:
# Очищаем от пробелов
df_full_flow['Unit'] = df_full_flow['Unit'].apply(lambda x: x.strip())

In [6]:
# Проверяем нет ли 7-ми значных кодов
df_full_flow['commodity_code'] = df_full_flow['commodity_code'].apply(lambda x: '0' + str(x) if len(str(x)) < 8 else str(x))

df_full_flow['seven'] = df_full_flow.commodity_code.apply(lambda x: 7 if len(str(x)) < 8 else 8)

df_full_flow.query('seven == 7')

,commodity_code,Commodity,year,trade_value,classification,period,reporter_code,name_country_source,aggregate_level,trade_flow_code,customs_proc_code,flag,plus,load_mark,update_date,Unit,qty,bool_border,seven


In [7]:
df_full_flow.query('name_country_source == "AFGHANISTAN" and period == "2023-03-01" and trade_flow_code == 1').trade_value.sum()

46960000.0

# Перечень всех кодов и названий продуктов

*Выполнить если он необходим. Для корректной сборки он не нужен.*

In [20]:
# Уникальное кол-во кодов продуктов
df_full_flow.commodity_code.nunique()

11553

In [21]:
# Уникальное количество названий продуктов
df_full_flow.Commodity.nunique()

10194

In [22]:
# Создаем датафрейм с уникальными названиями
df_corrected_by_сommodity = df_full_flow.groupby(['commodity_code', 'Commodity'], as_index=False).agg(n_customers=('year', 'count'))[['commodity_code', 'Commodity']]

In [23]:
# Валидируем его с родительским фреймом
df_corrected_by_сommodity.shape

(11553, 2)

In [24]:
df_corrected_by_сommodity.head(2)

,commodity_code,Commodity
0,01012100,PURE-BRED BREEDING HORSES
1,01012910,HORSES FOR POLO


In [25]:
# Записываем в файл
df_corrected_by_сommodity.to_excel('коды_и_названия.xlsx', index=False)

### [⬅ Навигация](#Навигация)

# Преобразование параметра qty датафрейма

In [8]:
# Справочник из БД с единицами измерения
query_deizm = f"""
        SELECT * FROM {name_table_or_bd["name_schema_reference"]}.{name_table_or_bd["name_table_reference"]}
        ORDER BY id ASC 
"""

df_deizm = pd.read_sql(query_deizm, con=engine)
df_deizm.rename(columns={'code': 'qty_unit_code','name_rus': 'Unit'}, inplace=True)
df_deizm = df_deizm[['qty_unit_code', 'Unit']]

In [19]:
# Словаврь для партнера
query_country_add = f"""

SELECT code, name_eng, name_country_source FROM {name_table_or_bd["name_schema_dl"]}.{name_table_or_bd["name_table_dl"]} 
WHERE source_id = 4

"""
df_country_add = pd.read_sql(query_country_add, con=engine)

df_country_add

,code,name_eng,name_country_source
0,897,Other 00,CANARY IS
1,132,Cabo Verde,CAPE VERDE IS
2,831,Bailiwick of Guernsey,GUERNSEY
3,832,Bailiwick of Jersey,JERSEY
4,849,US Miscellaneous Pacific Islands,PACIFIC IS
...,...,...,...
243,704,Viet Nam,VIETNAM SOC REP
244,850,US Virgin Islands,VIRGIN IS US
245,876,Wallis and Futuna Islands,WALLIS F IS
246,887,Yemen,YEMEN REPUBLC


In [10]:
dict_deizm = {}
# Заполняем словарь для перехода от ед. изм. itc к общепринятым
dict_deizm['NOS'] = 'штука'
dict_deizm['KGS'] = 'килограмм'
dict_deizm['SQM'] = 'квадратный метр'
dict_deizm['PRS'] = 'пара'
dict_deizm['LTR'] = 'литр'
dict_deizm['CTM'] = 'карат'
dict_deizm['CBM'] = 'кубический метр'
dict_deizm['kg'] = 'килограмм'
dict_deizm['MTR'] = 'метр'
dict_deizm['C/K'] = 'карат'
dict_deizm['KG'] = 'килограмм'
dict_deizm['UNT'] = 'штука'
dict_deizm['TKW'] = '1000 кВаттЧас'
dict_deizm['GIF'] = 'без размерности'
dict_deizm['THD'] = '1000 штук'
dict_deizm['TON'] = 'килограмм' #1000 * 1000 kg
dict_deizm['MTS'] = 'килограмм' #1000000 * 1000  kg
dict_deizm['0'] = 'без размерности'
dict_deizm['CRT'] = 'карат'
dict_deizm['\"'] = 'без размерности'

In [11]:
# Функция для приведения единиц измерения к единообразию
def fix_d_izm(unit, d_izm):
    if unit == 'MTS':
        return d_izm * 1000000000
    elif unit == 'TON':
        return d_izm * 1000000
    else:
        return d_izm * 1000

In [12]:
# df_full_flow.query("Unit == '\"'").to_excel('новые названия.xlsx', index=False)

In [13]:
df_full_flow['qty'] = df_full_flow.apply(lambda x: fix_d_izm(x.Unit, x.qty), axis=1)

In [14]:
df_full_flow['Unit'] = df_full_flow['Unit'].apply(lambda x: dict_deizm[x])

In [15]:
df_full_flow.head(3)

,commodity_code,Commodity,year,trade_value,classification,period,reporter_code,name_country_source,aggregate_level,trade_flow_code,customs_proc_code,flag,plus,load_mark,update_date,Unit,qty,bool_border,seven
0,07133190,BEANS OF THE SPP VIGNA RADIATA (L.) WILCZEK,2022,80000.0,HS,2022-05-01,699,AFGHANISTAN,8,1,C00,0,0,1,2023-07-11,килограмм,76000.0,not_none,8
1,07133990,OTHER DRIED LEGUMINUS VEGETABLES,2022,240000.0,HS,2022-05-01,699,AFGHANISTAN,8,1,C00,0,0,1,2023-07-11,килограмм,225000.0,not_none,8
2,08021100,ALMONDS FRSH OR DRIEDIN SHELL,2022,250000.0,HS,2022-05-01,699,AFGHANISTAN,8,1,C00,0,0,1,2023-07-11,килограмм,162450.0,not_none,8


In [16]:
# Корректируем значения для netweight
netweight_clear = []
for net_clear, unit in zip(list(df_full_flow.qty), list(df_full_flow.Unit)):
    if unit == 'килограмм':
        netweight_clear.append(net_clear)
    else:
        netweight_clear.append(0)
        
df_full_flow['netweight'] = netweight_clear

In [17]:
# Корреткируем значения для qty
netweight_fat = []
for net_fat, unit in zip(list(df_full_flow.qty), list(df_full_flow.Unit)):
    if unit != 'килограмм':
        netweight_fat.append(net_fat)
        
    else:
        netweight_fat.append(0)
        
df_full_flow['qty'] = netweight_fat

In [18]:
df_full_flow['region_code'] = 'NNNNN'

In [19]:
df_full_flow = df_full_flow.merge(df_deizm, on='Unit', how='left')

In [20]:
df_full_flow = df_full_flow.merge(df_country_add, on='name_country_source', how='left')

In [21]:
df_full_flow.isna().sum()

commodity_code         0
Commodity              0
year                   0
trade_value            0
classification         0
period                 0
reporter_code          0
name_country_source    0
aggregate_level        0
trade_flow_code        0
customs_proc_code      0
flag                   0
plus                   0
load_mark              0
update_date            0
Unit                   0
qty                    0
bool_border            0
seven                  0
netweight              0
region_code            0
qty_unit_code          0
code                   0
name_eng               0
dtype: int64

In [22]:
df_full_flow.rename(columns={'code': 'partner_code'}, inplace=True)                       

In [23]:
df_full_flow = df_full_flow[['classification', 'year', 'period', 'aggregate_level', 'trade_flow_code', 'region_code', 'reporter_code', 
                       'partner_code', 'customs_proc_code', 'commodity_code', 'qty_unit_code', 'qty', 'netweight', 
                       'trade_value', 'flag', 'plus', 'load_mark', 'update_date']]

In [24]:
df_full_flow.head(1)

,classification,year,period,aggregate_level,trade_flow_code,region_code,reporter_code,partner_code,customs_proc_code,commodity_code,qty_unit_code,qty,netweight,trade_value,flag,plus,load_mark,update_date
0,HS,2022,2022-05-01,8,1,NNNNN,699,4,C00,07133190,8,0.0,76000.0,80000.0,0,0,1,2023-07-11


In [25]:
df_full_flow.shape

(3170660, 18)

In [27]:
df_full_flow.trade_value.sum()

1629304810000.0

In [28]:
# Очищаем данные из БД
with engine.cursor() as cur:
    cur.execute(f'DELETE FROM {name_table_or_bd["name_schema_main"]}.{name_table_or_bd["name_table_main"]} WHERE reporter_code = 699')
    engine.commit()

In [12]:
df_count_value = pd.read_sql(f'SELECT COUNT(*) AS count_value FROM {name_table_or_bd["name_schema_main"]}.{name_table_or_bd["name_table_main"]} WHERE reporter_code = 699', 
                             con=engine)
сheck_zero = df_count_value.count_value[0]

In [30]:
if сheck_zero == 0:
    print(f'записей в таблице: {сheck_zero}, начинаем загрузку')
    df_full_flow.to_sql(name_table_or_bd["name_table_main"], con=conn, schema={name_table_or_bd["name_schema_main"], 
                                                                          if_exists='append', index=False)
else:
    print(f'записей в таблице: {сheck_zero}, произведите удаление лишних элементов')

записей в таблице: 0, начинаем загрузку


### [⬅ Навигация](#Навигация)